In [0]:
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
REBUILD_DATA=True

In [0]:
if REBUILD_DATA==True:
  import glob
  from tqdm.notebook import tqdm
  x=[]
  y=[]
  CATS='/content/drive/My Drive/dataset/training_set/cats/*.*'
  DOGS='/content/drive/My Drive/dataset/training_set/dogs/*.*'
  for f in tqdm(glob.glob(CATS)):
    img=cv2.imread(f,cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(28,28))
    x.append(img)
    y.append(0)
  for f in tqdm(glob.glob(DOGS)):
    img=cv2.imread(f,cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(28,28))
    x.append(img)
    y.append(1)


In [0]:
if REBUILD_DATA==True:
  import glob
  from tqdm.notebook import tqdm
  x_test=[]
  y_test=[]
  CATS='/content/drive/My Drive/dataset/test_set/cats/*.*'
  DOGS='/content/drive/My Drive/dataset/test_set/dogs/*.*'
  for f in tqdm(glob.glob(CATS)):
    img=cv2.imread(f,cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(28,28))
    x_test.append(img)
    y_test.append(0)
  for f in tqdm(glob.glob(DOGS)):
    img=cv2.imread(f,cv2.IMREAD_GRAYSCALE)
    img=cv2.resize(img,(28,28))
    x_test.append(img)
    y_test.append(1)

In [0]:
if REBUILD_DATA==True:
  from sklearn.model_selection import train_test_split
  x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=42)
  x_train=np.array(x_train)
  y_train=np.array(y_train)
  x_val=np.array(x_val)
  y_val=np.array(y_val)
  x_test=np.array(x_test)
  y_test=np.array(y_test)
  np.save('x_train.npy',x_train)
  np.save('y_train.npy',y_train)
  np.save('x_val.npy',x_val)
  np.save('y_val.npy',y_val)
  np.save('x_test.npy',x_test)
  np.save('y_test.npy',y_test)

In [0]:
x_train=np.load('x_train.npy',allow_pickle=True)
x_test=np.load('x_test.npy',allow_pickle=True)
y_train=np.load('y_train.npy',allow_pickle=True)
y_test=np.load('y_test.npy',allow_pickle=True)
x_val=np.load('x_val.npy',allow_pickle=True)
y_val=np.load('y_val.npy',allow_pickle=True)

In [0]:
x_train=x_train.reshape((6416,28,28,1))
x_val=x_val.reshape((1604,28,28,1))
x_test=x_test.reshape((2000,28,28,1))

In [0]:
x_train=x_train/255
x_val=x_val/255
x_test=x_test/255

In [0]:
if REBUILD_DATA==True:
  import tensorflow as tf
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                                  
    
  ])


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 50)                80050     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
if REBUILD_DATA==True:
  checkpoint=tf.keras.callbacks.ModelCheckpoint('CatVSDog.h5')
  r = model.fit(x_train, y_train, validation_data=(x_val, y_val),callbacks=[checkpoint],batch_size=32, epochs=20)

Epoch 1/20
201/201 [==============================] - 5s 27ms/step - loss: 0.6850 - accuracy: 0.5354 - val_loss: 0.6528 - val_accuracy: 0.6334
Epoch 2/20
201/201 [==============================] - 5s 27ms/step - loss: 0.6490 - accuracy: 0.6211 - val_loss: 0.6105 - val_accuracy: 0.6571
Epoch 3/20
201/201 [==============================] - 5s 27ms/step - loss: 0.6140 - accuracy: 0.6666 - val_loss: 0.5900 - val_accuracy: 0.6870
Epoch 4/20
201/201 [==============================] - 5s 27ms/step - loss: 0.5877 - accuracy: 0.6953 - val_loss: 0.5835 - val_accuracy: 0.6827
Epoch 5/20
201/201 [==============================] - 5s 27ms/step - loss: 0.5554 - accuracy: 0.7202 - val_loss: 0.5597 - val_accuracy: 0.7145
Epoch 6/20
201/201 [==============================] - 5s 27ms/step - loss: 0.5295 - accuracy: 0.7329 - val_loss: 0.5329 - val_accuracy: 0.7313
Epoch 7/20
201/201 [==============================] - 5s 27ms/step - loss: 0.5056 - accuracy: 0.7514 - val_loss: 0.5407 - val_accuracy: 0.7444

In [0]:
model.save('CatVSDogModel.h5')

In [0]:
#model=tf.keras.models.load_model('/content/Modells.h5')

In [0]:
predicted=model.predict(x_test)

In [0]:
c=0
for i in predicted:
  if i<0.5:
    predicted[c]=0
  else:
    predicted[c]=1
  c+=1



In [0]:

from sklearn.metrics import accuracy_score
print('Accuracy:',accuracy_score(y_test,predicted))


Accuracy: 0.744


In [0]:
# #import numpy as np
# xyz=np.arange(10)
# np.save('traikjga.npy',xyz)
# tr=np.load('trained_data.npy',allow_pickle=True)